In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI调整PEFT模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/tune_peft.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/tune_peft.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/tune_peft.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo">在Vertex AI Workbench中打开
    </a>
</table>

## 概述

本教程演示了如何使用Vertex AI来调整PEFT大型语言模型（LLM）并进行预测。该工作流通过使用训练数据集微调基础模型来提高模型的准确性。

了解更多关于[使用监督调整调整文本模型](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised)。

### 目标

在本教程中，您将学习如何使用 `Vertex AI LLM` 调整和部署一个PEFT大型语言模型。

本教程使用以下Google Cloud ML服务：

- `Vertex AI LLM`
- `Vertex AI Model Garden`
- `Vertex AI Prediction`

执行的步骤包括：

- 获取Vertex AI LLM模型。
- 调整模型。
  - 这将自动创建一个Vertex AI端点，并将模型部署到其中。
- 使用 `Vertex AI LLM` 进行预测。
- 使用 `Vertex AI Prediction` 进行预测。

### 模型

预训练的LLM模型是一种仅解码器的BISON模型，用于文本生成。

成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)，根据您预计的使用量生成费用估算。

## 安装

安装以下所需的包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform  "shapely<2.0.0"

### 仅限于 Colab：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type:"string"}

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作、取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查阅如何将云存储权限授予您的服务帐户，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

#### 服务账号

您可以使用服务账号来创建Vertex AI Pipeline任务。

In [ ]:
SERVICE_ACCOUNT = ""  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
        project_number = shell_output[0]
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI管道设置服务账号访问权限

运行以下命令，授予您的服务账号访问权限，以读取和写入在前一步中创建的存储桶中的管道工件。您只需要针对每个服务账号运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库并定义常数

In [ ]:
import google.cloud.aiplatform as aiplatform
from vertexai.preview.language_models import (TextGenerationModel,
                                              TuningEvaluationSpec)

## 为Python初始化Vertex AI SDK

为您的项目和相应的存储桶初始化Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 载入预训练模型

从Vertex AI LLM Model Garden载入预训练的BISON模型。

In [ ]:
model = TextGenerationModel.from_pretrained("google/text-bison@001")

model.list_tuned_model_names()

### 调整模型

接下来，您可以使用 `tune_model()` 方法调整模型，使用以下参数：

`training_data`: 用于调整模型的训练数据的 pandas Dataframe 或云存储位置。<br>
`learning_rate_multiplier`: 应用于推荐学习率的乘数。要使用推荐学习率，请使用1.0。<br>
`train_steps`: 用于模型调整的运行步数。批量大小根据调整位置而变化：<br>
- us-central1 的批量大小为8。
- europe-west4 的批量大小为24。<br>

如果训练数据集中有240个示例，在 europe-west4 中，处理整个数据集需要 240 / 24 = 10 步。在 us-central1 中，处理整个数据集需要 240 / 8 = 30 步。默认值为300。<br>

`tuning_job_location`: 应该运行调整作业的区域。支持的区域有：`us-central1` 和 `europe-west4`。<br>
`tuned_model_location`: 应部署经调整的模型的区域。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/model-evaluation/peft_eval_sample.jsonl {BUCKET_URI}/peft_eval_sample.jsonl
! gsutil cp gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl {BUCKET_URI}/peft_train_sample.jsonl

tuning_evaluation_spec = TuningEvaluationSpec(
    evaluation_data=f"{BUCKET_URI}/peft_eval_sample.jsonl",
    evaluation_interval=20,
    enable_early_stopping=True,
)
model.tune_model(
    training_data=f"{BUCKET_URI}/peft_train_sample.jsonl",
    # set to 1 for fast iteration and demo purpose. For this sample training dataset, we recommend at least 100 steps
    train_steps=100,
    learning_rate_multiplier=1.0,
    tuning_job_location="europe-west4",
    tuned_model_location="us-central1",  # Only us-central1 is supported
    model_display_name="test_model",
    tuning_evaluation_spec=tuning_evaluation_spec,
)

使用 Vertex AI LLM 接口中的 `predict()` 方法进行预测。

In [ ]:
prompt = "TRANSCRIPT: \nPROCEDURE PERFORMED: , Umbilical hernia repair.,PROCEDURE:,  After informed consent was obtained, the patient was brought to the operative suite and placed supine on the operating table.  The patient was sedated, and an adequate local anesthetic was administered using 1% lidocaine without epinephrine.  The patient was prepped and draped in the usual sterile manner.,A standard curvilinear umbilical incision was made, and dissection was carried down to the hernia sac using a combination of Metzenbaum scissors and Bovie electrocautery.  The sac was cleared of overlying adherent tissue, and the fascial defect was delineated.  The fascia was cleared of any adherent tissue for a distance of 1.5 cm from the defect.  The sac was then placed into the abdominal cavity and the defect was closed primarily using simple interrupted 0 Vicryl sutures.  The umbilicus was then re-formed using 4-0 Vicryl to tack the umbilical skin to the fascia.,The wound was then irrigated using sterile saline, and hemostasis was obtained using Bovie electrocautery.  The skin was approximated with 4-0 Vicryl in a subcuticular fashion.  The skin was prepped with benzoin, and Steri-Strips were applied.  A dressing was then applied.  All surgical counts were reported as correct.,Having tolerated the procedure well, the patient was subsequently taken to the recovery room in good and stable condition.\n\n LABEL: "

In [ ]:
print(model.predict(prompt))

获取已部署的Vertex AI端点资源

接下来，获取模型自动部署到的Vertex AI端点资源。

In [ ]:
endpoint = aiplatform.Endpoint(model._endpoint.resource_name)
print(endpoint)

使用Vertex AI预测进行预测

现在，使用Vertex AI预测接口中的`predict()`方法进行预测，使用以下参数：

- `instances`: 一个或多个要用于预测的实例列表。每个实例的格式为：
    - { "content": 输入的文本 }
- `parameters`: 传递给模型的预测方法的参数。对应的示例为默认值。

In [ ]:
endpoint.predict(
    instances=[{"prompt": prompt}],
    parameters={
        "temperature": 0.0,
        "maxDecodeSteps": 128,
        "topP": 0.95,
        "topK": 40,
    },
)

### 运行后调整评估

请注意，`ground_truth_data` 的格式应该是一个 JSONL 文件，其中每一行都是以下格式的一个 json：

```
{
  "prompt": "您的输入/提示文本",
  "ground_truth": "您的真实输出文本"
}
```

- "prompt" 对应于训练数据集中的 "input_text"。这是为了进行批量预测。
- "ground_truth" 对应于训练数据集中的 "output_text"。这是用于评估的。

In [ ]:
from vertexai.preview.language_models import EvaluationTextGenerationSpec

tuned_model = model

# Uncomment the following to load a tuned model if the tuning session is broken
# tuned_model = TextGenerationModel.from_pretrained("google/text-bison@001")
# tuned_model.get_tuned_model(f'projects/{PROJECT_ID}/locations/us-central1/models/3890975937629519872')

# Text generation example
evaluation_task_spec = EvaluationTextGenerationSpec(
    ground_truth_data=[
        "gs://cloud-samples-data/vertex-ai/model-evaluation/peft_test_sample.jsonl"
    ]
)

tuned_model.evaluate(task_spec=evaluation_task_spec)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

endpoint.undeploy_all()
endpoint.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}